In [1]:
sample_sizes = [100,300,500,800,1000,3000,5000]
weights = [0.08]
min_s = 0.6
min_c = 0.9
origin_data = "sachs"
bif_data = origin_data + ".bif"
sample_csv_data = origin_data + ".csv"
final_csv_data = "final_03071"+ origin_data + ".csv"

In [2]:
from pgmpy.estimators import StructureScore
from scipy.special import gammaln
from math import lgamma, log
class BDeuScore_pt(StructureScore):
    def __init__(self, data, priori_subgraph, black_priori_subgraph, weight, equivalent_sample_size=10, **kwargs):
        """
        Class for Bayesian structure scoring for BayesianModels with Dirichlet priors.
        The BDeu score is the result of setting all Dirichlet hyperparameters/pseudo_counts to
        `equivalent_sample_size/variable_cardinality`.
        The `score`-method measures how well a model is able to describe the given data set.
        Parameters
        ----------
        data: pandas DataFrame object
            datafame object where each column represents one variable.
            (If some values in the data are missing the data cells should be set to `numpy.NaN`.
            Note that pandas converts each column containing `numpy.NaN`s to dtype `float`.)
        equivalent_sample_size: int (default: 10)
            The equivalent/imaginary sample size (of uniform pseudo samples) for the dirichlet hyperparameters.
            The score is sensitive to this value, runs with different values might be useful.
        state_names: dict (optional)
            A dict indicating, for each variable, the discrete set of states (or values)
            that the variable can take. If unspecified, the observed values in the data set
            are taken to be the only possible states.
        complete_samples_only: bool (optional, default `True`)
            Specifies how to deal with missing data, if present. If set to `True` all rows
            that contain `np.Nan` somewhere are ignored. If `False` then, for each variable,
            every row where neither the variable nor its parents are `np.NaN` is used.
            This sets the behavior of the `state_count`-method.
        References
        ---------
        [1] Koller & Friedman, Probabilistic Graphical Models - Principles and Techniques, 2009
        Section 18.3.4-18.3.6 (esp. page 806)
        [2] AM Carvalho, Scoring functions for learning Bayesian networks,
        http://www.lx.it.pt/~asmc/pub/talks/09-TA/ta_pres.pdf
        """
        self.equivalent_sample_size = equivalent_sample_size
        self.priori_subgraph = priori_subgraph
        self.black_priori_subgraph = black_priori_subgraph
        self.weight = weight
        super(BDeuScore_pt, self).__init__(data, **kwargs)

    def local_score(self, variable, parents):
        'Computes a score that measures how much a \
        given variable is "influenced" by a given list of potential parents.'

        var_states = self.state_names[variable]
        var_cardinality = len(var_states)
        state_counts = self.state_counts(variable, parents)
        num_parents_states = float(state_counts.shape[1])

        counts = np.asarray(state_counts)
        log_gamma_counts = np.zeros_like(counts, dtype=np.float_)

        alpha = self.equivalent_sample_size / num_parents_states
        beta = self.equivalent_sample_size / counts.size

        # Compute log(gamma(counts + beta))
        gammaln(counts + beta, out=log_gamma_counts)

        # Compute the log-gamma conditional sample size
        log_gamma_conds = np.sum(counts, axis=0, dtype=np.float_)
        gammaln(log_gamma_conds + alpha, out=log_gamma_conds)
        
        s = 0
        #print("parents:")
        #print(parents)
        for parent in parents:
            edge = (parent,variable)
            if edge in self.priori_subgraph:
                s += self.weight
                #print(edge, s, counts)
            elif edge in self.black_priori_subgraph:
                s -= self.weight

        score = (
            np.sum(log_gamma_counts)
            - np.sum(log_gamma_conds)
            + num_parents_states * lgamma(alpha)
            - counts.size * lgamma(beta)
            + counts.size * s
        )
        #print(lgamma(beta))
        return score

/bin/anaconda3/envs/beyes/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
def hmdis(model, model1):
    rev = 0
    sam = 0
    dis = 0
    model_len = len(model)
    model1_len = len(model1)
    for itemset in model:
        itemset1 = (itemset[1], itemset[0])
        if itemset in model1:
            #print(itemset)
            sam += 1
        elif itemset1 in model1:
            rev += 1
    dis = model_len + model1_len - 2*sam - rev
    return dis

In [4]:
def data_processer(sample_size):
    gum.generateCSV(bn_o,os.path.join("/root",sample_csv_data),sample_size,True)
    data1=pd.read_csv("/root/" + sample_csv_data)
    for index, row in data1.iteritems():
        if data1[index].max()>1:
            #print(index)
            count0=0
            count1=0
            for indexrow, row in data1.iterrows():
                #print(row)
                if row[index]<data1[index].max()/2:
                    row[index]=0
                    count0=count0+1
                else:
                    row[index]=1
                    count1=count1+1
            #print(count0,count1)
        else:
            #print(index)
            count0=0
            count1=0
            for indexrow, row in data1.iterrows():
                #print(row)
                if row[index]==0:
                    count0=count0+1
                else:
                    count1=count1+1
            #print(count0,count1)
    return data1

In [5]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
def min_frequent_item(data1):
    frequent_itemsets = apriori(data1, min_support=min_s, use_colnames=True)
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    return frequent_itemsets

In [6]:
import itertools
def min_ass_subgraph(frequent_itemsets):
    frequent_itemsets_ass=frequent_itemsets[frequent_itemsets['length']<=2]
    rules = association_rules(frequent_itemsets_ass, min_threshold=0.7)
    rules = rules[rules['confidence']>=min_c]
    #print(rules)
    ass_subgraph = []
    for index,rule in rules.iterrows():
        x=(list(rule['antecedents'])[0],list(rule['consequents'])[0])
        ass_subgraph.append(x)
    print(ass_subgraph)
    return ass_subgraph

In [7]:
def min_max_fre_itemsets(frequent_itemsets):
    frequent_itemsets_new=frequent_itemsets[frequent_itemsets['length']>1]
    max_fre_itemsets = []
    flag = False
    frequent_itemsets_new = frequent_itemsets_new.sort_values(by="length", ascending=False)
    for index,itemset in frequent_itemsets_new.iterrows():
        #print(itemset['itemsets'])
        for max_fre_itemset in max_fre_itemsets:
            if itemset['itemsets'].issubset(max_fre_itemset):
                flag = True
                break
        if not flag:
            max_fre_itemsets.append(itemset['itemsets'])
            #print(max_fre_itemsets)
        else:
            flag = False
    #for x in max_fre_itemsets[0]:
        #print(x)
    print(max_fre_itemsets)
    return max_fre_itemsets

In [8]:
def priori_structrue_PC(max_fre_itemsets,ass_subgraph):
    data=pd.read_csv("/root/" + sample_csv_data)
    priori_subgraph = []
    for i in range(0,len(max_fre_itemsets)):
        node=[]
        df_node=pd.DataFrame()
        for x in max_fre_itemsets[i]:
            node.append(x)
        #print(node)
        df_node=data[node]
        c = PC(df_node)
        #print(df_node)
        best_model = c.estimate()
        for edge in best_model.edges():
            if not edge in priori_subgraph:
                print(edge)
                if edge in ass_subgraph:
                    priori_subgraph.append(edge)
    print(priori_subgraph)
    return priori_subgraph

In [9]:
def cal_black_priori(max_fre_itemsets,priori_subgraph,ass_subgraph):
    black_priori_subgraph = []
    for max_fre_itemset in max_fre_itemsets:
        for i in itertools.permutations(max_fre_itemset, 2):
            j = (i[1], i[0])
            #if not i in priori_subgraph and not j in priori_subgraph:
            if not i in priori_subgraph:
                if not i in ass_subgraph:
                    black_priori_subgraph.append(i)
    black_priori_subgraph = list(set(black_priori_subgraph))
    print(black_priori_subgraph)
    return black_priori_subgraph

In [10]:
def get_origin_model():
    gum.about()
    bn_o=gum.loadBN(os.path.join("/root/beyes",bif_data))
    data=pd.read_csv("/root/" + sample_csv_data)
    bdeu = bds(data)
    names_o=bn_o.names()
    arcs_o=bn_o.arcs()
    d_o=[]
    for (u,v) in arcs_o:
        f=(names_o[u],names_o[v])
        d_o.append(f)

    modelo = BayesianModel(d_o)
    return modelo

In [11]:
def get_final_data(final_data, origin_model, best_model_priori, best_model_origin, sample_size, weight):
    data=pd.read_csv("/root/" + sample_csv_data)
    bdeu = bds(data)
    tmp_data=pd.DataFrame(columns=("sample_size","weight","best_priori_score","best_origin_score","origin_score","priori_dis","origin_dis"))
    tmp_data["origin_score"] = [bdeu.score(origin_model)]
    tmp_data["best_priori_score"] = [bdeu.score(best_model_priori)]
    tmp_data["best_origin_score"] = [bdeu.score(best_model_origin)]
    origin_edge = origin_model.edges()
    best_priori_edge = best_model_priori.edges()
    best_origin_edge = best_model_origin.edges()
    tmp_data["priori_dis"] = [hmdis(origin_model.edges(),best_model_priori.edges())]
    tmp_data["origin_dis"] = [hmdis(origin_model.edges(),best_model_origin.edges())]
    tmp_data["sample_size"] = [sample_size]
    tmp_data["weight"] = [weight]
    final_data=final_data.append(tmp_data)
    return final_data

In [12]:
def get_final_data_optimize(final_data, origin_model, best_model_prioris, best_model_origin, sample_size):
    data=pd.read_csv("/root/" + sample_csv_data)
    bdeu = bds(data)
    tmp_data=pd.DataFrame(columns=("sample_size","weight","best_priori_score","best_origin_score","origin_score","priori_dis","origin_dis"))
    index=0
    for weight in weights:
        tmp_data["origin_score"] = [bdeu.score(origin_model)]
        tmp_data["best_priori_score"] = [bdeu.score(best_model_prioris[index])]
        tmp_data["best_origin_score"] = [bdeu.score(best_model_origin)]
        origin_edge = origin_model.edges()
        best_priori_edge = best_model_prioris[index].edges()
        best_origin_edge = best_model_origin.edges()
        tmp_data["priori_dis"] = [hmdis(origin_model.edges(),best_model_prioris[index].edges())]
        tmp_data["origin_dis"] = [hmdis(origin_model.edges(),best_model_origin.edges())]
        tmp_data["sample_size"] = [sample_size]
        tmp_data["weight"] = [weight]
        final_data=final_data.append(tmp_data)
        #print(best_model_prioris[index].edges())
        #print(best_model_origin.edges())
        index +=1
    return final_data

In [13]:
def final_test(sample_size,weight,final_data):
    data1 = data_processer(sample_size)
    frequent_itemsets = min_frequent_item(data1)
    ass_subgraph = min_ass_subgraph(frequent_itemsets)
    max_fre_itemsets = min_max_fre_itemsets(frequent_itemsets)
    priori_subgraph = priori_structrue_PC(max_fre_itemsets, ass_subgraph)
    black_priori_subgraph = cal_black_priori(max_fre_itemsets, priori_subgraph,ass_subgraph)
    origin_model = get_origin_model()
    data=pd.read_csv("/root/" + sample_csv_data)
    hc = HillClimbSearch(data)
    best_model_priori = hc.estimate(scoring_method=BDeuScore_pt(data, priori_subgraph, black_priori_subgraph,weight))
    best_model_origin = hc.estimate(scoring_method=bds(data))
    final_data=get_final_data(final_data, origin_model, best_model_priori, best_model_origin, sample_size, weight)
    return final_data

In [14]:
def final_test_optimize(sample_size,final_data):
    data1 = data_processer(sample_size)
    frequent_itemsets = min_frequent_item(data1)
    ass_subgraph = min_ass_subgraph(frequent_itemsets)
    max_fre_itemsets = min_max_fre_itemsets(frequent_itemsets)
    priori_subgraph = priori_structrue_PC(max_fre_itemsets, ass_subgraph)
    black_priori_subgraph = cal_black_priori(max_fre_itemsets, priori_subgraph,ass_subgraph)
    origin_model = get_origin_model()
    data=pd.read_csv("/root/" + sample_csv_data)
    hc = HillClimbSearch(data)
    best_model_prioris=[]
    for weight in weights:
        best_model_priori = hc.estimate(scoring_method=BDeuScore_pt(data, priori_subgraph, black_priori_subgraph,weight))
        best_model_prioris.append(best_model_priori)
    best_model_origin = hc.estimate(scoring_method=bds(data))
    final_data=get_final_data_optimize(final_data, origin_model, best_model_prioris, best_model_origin, sample_size)
    return final_data

In [15]:
import matplotlib.pyplot as plt
import os
import pyAgrum as gum
import pandas as pd
import numpy as np
from pgmpy.estimators import BDeuScore as bds
from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import PC

gum.about()
bn_o=gum.loadBN(os.path.join("/root/beyes",bif_data))
final_data=pd.DataFrame(columns=("sample_size","weight","best_priori_score","best_origin_score","origin_score","priori_dis","origin_dis"))
for sample_size in sample_sizes:
    i=0
    for i in range(0,15):
        final_data=final_test_optimize(sample_size,final_data)
        i=i+1
    final_data.to_csv(final_csv_data)
#final_data

pyAgrum version 0.19.0
(c) 2015-2020 Pierre-Henri Wuillemin, Christophe Gonzales, Lionel Torti

    This is free software; see the source code for copying conditions.
    There is ABSOLUTELY NO WARRANTY; not even for MERCHANTABILITY or
    FITNESS FOR A PARTICULAR PURPOSE.  For details, see 'pyAgrum.warranty'.
    
 /root/sachs.csv : [ ######################################################### ] 100%
Log2-Likelihood : -975.9326243738866


  0%|          | 0/1000000 [00:00<?, ?it/s]

[('PKA', 'Erk'), ('PIP3', 'Erk')]
[frozenset({'Erk', 'PKA'}), frozenset({'Erk', 'PIP3'})]
('Erk', 'PKA')
[]
[('Erk', 'PIP3'), ('Erk', 'PKA')]
pyAgrum version 0.19.0
(c) 2015-2020 Pierre-Henri Wuillemin, Christophe Gonzales, Lionel Torti

    This is free software; see the source code for copying conditions.
    There is ABSOLUTELY NO WARRANTY; not even for MERCHANTABILITY or
    FITNESS FOR A PARTICULAR PURPOSE.  For details, see 'pyAgrum.warranty'.
    


  0%|          | 15/1000000 [00:03<55:47:28,  4.98it/s]


 /root/sachs.csv : [ ######################################################### ] 100%
Log2-Likelihood : -1028.274162149896


...

  0%|          | 18/1000000 [00:03<49:06:26,  5.66it/s]


 /root/sachs.csv : [ ######################################################### ] 100%
Log2-Likelihood : -51763.470527963924


  0%|          | 0/1000000 [00:00<?, ?it/s]

[('PKA', 'Erk')]
[frozenset({'Erk', 'PIP3'}), frozenset({'PKA', 'PIP3'}), frozenset({'Erk', 'PKA'})]
('Erk', 'PKA')
[]
[('PIP3', 'Erk'), ('Erk', 'PKA'), ('PIP3', 'PKA'), ('Erk', 'PIP3'), ('PKA', 'PIP3')]
pyAgrum version 0.19.0
(c) 2015-2020 Pierre-Henri Wuillemin, Christophe Gonzales, Lionel Torti

    This is free software; see the source code for copying conditions.
    There is ABSOLUTELY NO WARRANTY; not even for MERCHANTABILITY or
    FITNESS FOR A PARTICULAR PURPOSE.  For details, see 'pyAgrum.warranty'.
    


  0%|          | 19/1000000 [00:03<50:03:15,  5.55it/s]
